In [1]:
import csv
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shapefile
import pickle
import os

In [206]:
#read link data
link_path = os.path.join("transportation_data","wislr-data","wislr_rdwy_link.csv")
link_data = pd.read_csv(link_path)

#read link time
time_path = os.path.join("link_time.csv")
time_data = pd.read_csv(time_path,index_col=0)

#read link name
name_path = os.path.join("transportation_data","wislr-data","wislr_rdwy_rte_link.csv")
name_data = pd.read_csv(name_path)

In [218]:
# add road name and link id
time_data['RDWY_LINK_ID'] = link_data['RDWY_LINK_ID']


In [224]:
# # add roadway rte id
# RDWY_RTE_ID = []
# for i in list(time_data['RDWY_LINK_ID']):
#     RDWY_RTE_ID.append(name_data.loc[name_data['RDWY_LINK_ID'] == i,'RDWY_RTE_ID'].iloc[0])

# time_data["RDWY_RTE_ID"] = RDWY_RTE_ID

In [227]:
time_data['FROM'] = time_data['FROM'].astype(str)
time_data['TO'] = time_data['TO'].astype(str)

In [225]:
time_data

,FROM,TO,TIME,RDWY_LINK_ID,RDWY_RTE_ID
0,1661200,1660820,116.171429,4818580,400333
1,1661678,1661705,6.320000,4819008,396810
2,1661704,1661705,10.560000,4819009,396788
3,1661723,1661697,23.240000,4819014,396769
4,1661705,1661723,10.560000,4819015,396736
...,...,...,...,...,...
3127,1663210,1663276,25.075000,5559301,395130
3128,1661558,1661551,9.057143,5559302,395130
3129,1661551,1661558,9.057143,5559529,395003
3130,2016715,2005025,31.657143,5559530,395003


In [214]:
name_data.head()

,WISLR_PTY_ID,WISLR_PTY_TYCD,WISLR_PTY_NM,RDWY_RTE_ID,SORT_NM,RDWY_LINK_ID,RWLK_STRT_OSET,RWLK_END_OSET,RWRL_SORT,BASC_RTE_FL,DIR_INDC
0,333,T,CROSS PLAINS,394349,AIRPORT RD,4840537,264,528,100,Y,>
1,333,T,CROSS PLAINS,394349,AIRPORT RD,5253200,0,132,200,Y,>
2,333,T,CROSS PLAINS,394349,AIRPORT RD,5253202,197,7762,300,Y,>
3,333,T,CROSS PLAINS,394349,AIRPORT RD,5253201,0,7565,400,N,<
4,333,T,CROSS PLAINS,394349,AIRPORT RD,5253199,185,317,500,N,<


In [222]:
name_data

,WISLR_PTY_ID,WISLR_PTY_TYCD,WISLR_PTY_NM,RDWY_RTE_ID,SORT_NM,RDWY_LINK_ID,RWLK_STRT_OSET,RWLK_END_OSET,RWRL_SORT,BASC_RTE_FL,DIR_INDC
0,333,T,CROSS PLAINS,394349,AIRPORT RD,4840537,264,528,100,Y,>
1,333,T,CROSS PLAINS,394349,AIRPORT RD,5253200,0,132,200,Y,>
2,333,T,CROSS PLAINS,394349,AIRPORT RD,5253202,197,7762,300,Y,>
3,333,T,CROSS PLAINS,394349,AIRPORT RD,5253201,0,7565,400,N,<
4,333,T,CROSS PLAINS,394349,AIRPORT RD,5253199,185,317,500,N,<
...,...,...,...,...,...,...,...,...,...,...,...
3376,379,C,MIDDLETON,702113,BREAKERS ROCK RD,5422167,0,947,400,N,<
3377,379,C,MIDDLETON,744117,CROSSOVER USH 00012,5519750,0,72,100,Y,>
3378,379,C,MIDDLETON,744117,CROSSOVER USH 00012,5519749,0,72,200,N,<
3379,379,C,MIDDLETON,756816,SCHWARTZ RD,5035654,0,898,99,Y,>


In [205]:
time_data.head()

,FROM,TO,TIME,RDWY_LINK_ID
0,1661200,1660820,116.171429,4818580
1,1661678,1661705,6.320000,4819008
2,1661704,1661705,10.560000,4819009
3,1661723,1661697,23.240000,4819014
4,1661705,1661723,10.560000,4819015


In [10]:
link_data.head()

,RDWY_LINK_ID,REF_SITE_FROM_ID,REF_SITE_TO_ID,LCM_FROM_TO_DIS,RDWY_LINK_RSLN
0,4818580,1661200,1660820,4066,52.799999
1,4819008,1661678,1661705,158,52.799999
2,4819009,1661704,1661705,264,52.799999
3,4819014,1661723,1661697,581,52.799999
4,4819015,1661705,1661723,264,52.799999


In [207]:
name_data.head()

,WISLR_PTY_ID,WISLR_PTY_TYCD,WISLR_PTY_NM,RDWY_RTE_ID,SORT_NM,RDWY_LINK_ID,RWLK_STRT_OSET,RWLK_END_OSET,RWRL_SORT,BASC_RTE_FL,DIR_INDC
0,333,T,CROSS PLAINS,394349,AIRPORT RD,4840537,264,528,100,Y,>
1,333,T,CROSS PLAINS,394349,AIRPORT RD,5253200,0,132,200,Y,>
2,333,T,CROSS PLAINS,394349,AIRPORT RD,5253202,197,7762,300,Y,>
3,333,T,CROSS PLAINS,394349,AIRPORT RD,5253201,0,7565,400,N,<
4,333,T,CROSS PLAINS,394349,AIRPORT RD,5253199,185,317,500,N,<


In [229]:
arc_data = time_data.copy()

In [230]:
arc_data.head()

,FROM,TO,TIME,RDWY_LINK_ID,RDWY_RTE_ID
0,1661200,1660820,116.171429,4818580,400333
1,1661678,1661705,6.320000,4819008,396810
2,1661704,1661705,10.560000,4819009,396788
3,1661723,1661697,23.240000,4819014,396769
4,1661705,1661723,10.560000,4819015,396736


In [228]:
arc_data = time_data.copy()
arc_data['use'] = 1
arc_data.loc[arc_data["FROM"]==arc_data["TO"] ,'use'] = 0

#change data type to string
arc_data['FROM'] = arc_data['FROM'].astype(str)
arc_data['TO'] = arc_data['TO'].astype(str)
# start processing nodes

# find all the unique nodes
nodes = set(list(arc_data["FROM"]) + list(arc_data["TO"]))
interesction_index = 1

loopTrack = 1
# for each node, start generating buffer node acccording to the in and out arc
for N in nodes:
    new_arcs = []
    out_N = list(arc_data[(arc_data['FROM'] == N)&(arc_data['use'] == 1)]['TO'])
    out_N_time = list(arc_data[(arc_data['FROM'] == N)&(arc_data['use'] == 1)]['TIME'])
    out_N_id = list(arc_data[(arc_data['FROM'] == N)&(arc_data['use'] == 1)]['RDWY_LINK_ID'])
    in_N = list(arc_data[(arc_data['TO'] == N)&(arc_data['use'] == 1)]['FROM'])
    in_N_time = list(arc_data[(arc_data['TO'] == N)&(arc_data['use'] == 1)]['TIME'])
    in_N_id = list(arc_data[(arc_data['TO'] == N)&(arc_data['use'] == 1)]['RDWY_LINK_ID'])


    new_nodes = list(set(out_N + in_N))
    new_nodes_dict = {}
    for i in range(len(new_nodes)):
        new_nodes_dict[new_nodes[i]] = str(N) + "_"+ str(i + 1) 
    
    # connnect new nodes with original nodes
    for index in range(len(out_N)):
        new_arcs.append([new_nodes_dict[out_N[index]],out_N[index],out_N_time[index],out_N_id[index],1])
        
    for index in range(len(in_N)):
        new_arcs.append([in_N[index],new_nodes_dict[in_N[index]],in_N_time[index],in_N_id[index],1])
        
    # connect new nodes with each other           
    for i in in_N:
        for o in out_N:
            # do not allow turn around
            if i != o:
                new_arcs.append([new_nodes_dict[i],new_nodes_dict[o],-1,'intersection_'+str(interesction_index),1])
                interesction_index += 1
                
    # disconnect original arcs
    arc_data.loc[(arc_data['FROM'] == N) | (arc_data['TO'] == N) , 'use'] = 0
    
    # add arcs to arc set
    new_arcs = pd.DataFrame(new_arcs, columns=["FROM","TO","TIME","RDWY_LINK_ID","use"])
    arc_data = arc_data.append(new_arcs,ignore_index=True)
    

# connect alias node according to in and out arcs
    # take note of the original link id and traveling time
    # take note of the length of each link
    
arc_data = arc_data[arc_data['use'] == 1]

arc_data = arc_data.reset_index(drop=True)

In [172]:
#random_N = random.choice(list(nodes))

In [203]:
#arc_data[arc_data['FROM'].contains("1661200")]
arc_data[(arc_data['FROM'].str.contains("3"))| (arc_data['TO'].str.contains("_"))]
#arc_data[arc_data['use'] == 1]

,FROM,TO,TIME,RDWY_LINK_ID,use
6,1662528_1,1662528_2,-1.000000,intersection_7,1
7,1662528_1,1662528_3,-1.000000,intersection_8,1
8,1662528_2,1662528_1,-1.000000,intersection_9,1
9,1662528_2,1662528_3,-1.000000,intersection_10,1
17,1662147_4,1662147_2,-1.000000,intersection_18,1
...,...,...,...,...,...
9034,1667498_2,1667498_1,-1.000000,intersection_5909,1
9035,1667498_2,1667498_3,-1.000000,intersection_5910,1
9036,1667498_1,1667498_2,-1.000000,intersection_5911,1
9037,1667498_1,1667498_3,-1.000000,intersection_5912,1


In [192]:
time_data[(time_data["FROM"]=="2157021") |(time_data["TO"]=="2157021") ]

,FROM,TO,TIME,RDWY_LINK_ID
2739,2157021,2157021,14.742857,5260983
2752,2129751,2157021,4.542857,5285778
2753,2157021,2129751,4.542857,5285779


In [181]:
arc_data[(- arc_data['FROM'].str.contains("_"))& (- arc_data['TO'].str.contains("_"))]


,FROM,TO,TIME,RDWY_LINK_ID,use


In [176]:
test_list = []
for i in range(len(from_list)):
    test_list.append((from_list[i],to_list[i]))

In [185]:
for i in from_list:
    if "_" not in i:
        print(i)

1662248
2030922
2157021
1667287


In [186]:
for i in to_list:
    if "_" not in i:
        print(i)

1662248
2030922
2157021
1667287
